## Importing The Libraries

In [64]:
import numpy as np
import pandas as pd
import datetime
import scipy

import py_vollib.black_scholes as bs
import py_vollib.black_scholes.greeks.analytical as greeks
import py_vollib.black_scholes.implied_volatility as iv

 # To calculate greeks
import mibian

 # For plotting
import matplotlib.pyplot as plt

import warnings
warnings.simplefilter("ignore")

In [65]:
# Assigning Weight for constituents and lot sizes
HDFCBANK_Wt = 0.333
HDFCBANK_Lot_Size = 500

ICICIBANK_Wt = 0.173
ICICIBANK_Lot_Size = 2500

KOTAKBANK_Wt = 0.123
KOTAKBANK_Lot_Size = 800

SBIN_Wt = 0.102
SBIN_Lot_Size = 3000

AXISBANK_Wt = 0.08
AXISBANK_Lot_Size = 1200

BankNifty_Wt = 1.0
BankNifty_Lot_Size = 40

In [66]:
# Read The data
def read_data(inst_name):
    opt = pd.read_csv(inst_name + ".csv")
    fut = pd.read_csv('BankNifty_Options_Data.csv')
    opt = opt[opt['Open Int'] != 0]
    opt = opt[['Symbol', 'Date', 'Expiry', 'Option Type',
               'Strike Price', 'Close', 'futures_price']]
    return opt

BankNifty_Opt = read_data('BankNifty_Options_Data')
BankNifty_Opt = BankNifty_Opt[(BankNifty_Opt.Symbol == 'BANKNIFTY')]
BankNifty_Opt.head()

,Symbol,Date,Expiry,Option Type,Strike Price,Close,futures_price
0,BANKNIFTY,01-Dec-17,28-Dec-17,CE,22400.0,2956.0,25259.65
1,BANKNIFTY,01-Dec-17,28-Dec-17,CE,27500.0,8.0,25259.65
6,BANKNIFTY,01-Dec-17,28-Dec-17,CE,23000.0,2249.4,25259.65
7,BANKNIFTY,01-Dec-17,28-Dec-17,CE,23100.0,2266.7,25259.65
8,BANKNIFTY,01-Dec-17,28-Dec-17,CE,23200.0,2320.0,25259.65


In [67]:
# Calculate Time to expiry
def time_to_expiry(opt):
    opt.Expiry = pd.to_datetime(opt.Expiry)
    opt.Date = pd.to_datetime(opt.Date)
    opt['time_to_expiry'] = (opt.Expiry - opt.Date).dt.days
    return opt


BankNifty_Opt = time_to_expiry(BankNifty_Opt)
BankNifty_Opt.head()

,Symbol,Date,Expiry,Option Type,Strike Price,Close,futures_price,time_to_expiry
0,BANKNIFTY,2017-12-01,2017-12-28,CE,22400.0,2956.0,25259.65,27
1,BANKNIFTY,2017-12-01,2017-12-28,CE,27500.0,8.0,25259.65,27
6,BANKNIFTY,2017-12-01,2017-12-28,CE,23000.0,2249.4,25259.65,27
7,BANKNIFTY,2017-12-01,2017-12-28,CE,23100.0,2266.7,25259.65,27
8,BANKNIFTY,2017-12-01,2017-12-28,CE,23200.0,2320.0,25259.65,27


In [68]:
# determine At-The-Money Strike Price
def atm_strike_price(opt):
    opt['strike_distance'] = np.abs(opt.futures_price - opt['Strike Price'])
    df = opt.groupby(['Date'])['strike_distance'].min().to_frame()
    df.index.column = 0
    opt = pd.merge(opt, df)
    opt = opt[(np.abs(opt.futures_price - opt['Strike Price'])
               == opt.strike_distance)]
    opt = opt.drop('strike_distance', 1)
    opt = opt.drop_duplicates(subset=['Date', 'Expiry', 'Option Type'])
    return opt


full_BankNifty_opt = BankNifty_Opt
BankNifty_Opt = atm_strike_price(BankNifty_Opt)
BankNifty_Opt.tail()

,Symbol,Date,Expiry,Option Type,Strike Price,Close,futures_price,time_to_expiry
199,BANKNIFTY,2017-10-23,2017-10-26,PE,24100.0,147.70,24123.35,3
200,BANKNIFTY,2017-10-24,2017-10-26,CE,24200.0,152.45,24237.55,2
201,BANKNIFTY,2017-10-24,2017-10-26,PE,24200.0,112.75,24237.55,2
202,BANKNIFTY,2017-10-25,2017-10-26,CE,25000.0,112.35,24981.20,1
203,BANKNIFTY,2017-10-25,2017-10-26,PE,25000.0,129.50,24981.20,1


In [69]:
# Calculate PnL from Long Straddle
def daily_pnl(opt, full_opt):
    opt['next_day_close'] = np.nan
    opt.sort_values('Date', inplace=True, ascending=True)
    for i in range(0, len(opt)-2):
        strike_price = opt.iloc[i]['Strike Price']
        trade_date = opt.iloc[i]['Date']
        next_trading_date = opt[(opt.Date > trade_date)
                                & (opt.Date <= trade_date + datetime.timedelta(days=20)
                                   )].iloc[0]['Date']
        option_type = opt.iloc[i]['Option Type']

        if opt.iloc[i]['time_to_expiry'] != 0:
            opt.iloc[i, opt.columns.get_loc('next_day_close')] = full_opt[(full_opt['Strike Price'] == strike_price) &
                                                                          (full_opt['Date'] == next_trading_date) &
                                                                          (full_opt['Option Type']
                                                                           == option_type)
                                                                          ].iloc[0]['Close']
        else:
            # This is done because on expiry day the next day price doesn't exists
            opt.iloc[i, opt.columns.get_loc(
                'next_day_close')] = opt.iloc[i]['Close']

    opt['daily_straddle_pnl'] = opt.next_day_close - opt.Close
    return opt


BankNifty_Opt = daily_pnl(BankNifty_Opt, full_BankNifty_opt)
BankNifty_Opt.head()

,Symbol,Date,Expiry,Option Type,Strike Price,Close,futures_price,time_to_expiry,next_day_close,daily_straddle_pnl
118,BANKNIFTY,2017-07-28,2017-08-31,CE,24900.0,382.00,24934.25,34,509.65,127.65
119,BANKNIFTY,2017-07-28,2017-08-31,PE,24900.0,355.30,24934.25,34,284.35,-70.95
120,BANKNIFTY,2017-07-31,2017-08-31,CE,25100.0,396.60,25135.05,31,420.00,23.40
122,BANKNIFTY,2017-07-31,2017-08-31,PE,25100.0,367.45,25135.05,31,326.70,-40.75
128,BANKNIFTY,2017-08-01,2017-08-31,CE,25200.0,366.15,25195.35,30,320.60,-45.55


In [70]:
# def implied_volatility_options(opt):
#     opt['IV'] = np.nan
#     opt = opt.iloc[:3]
#     opt.loc[(opt.time_to_expiry == 0), 'time_to_expiry'] = 0.0000001
#     for i in range(0, len(opt)):
#         if opt.iloc[i]['Option Type'] == 'CE':
#             opt.iloc[i, opt.columns.get_loc('IV')] = mibian.BS([opt.iloc[i]['futures_price'],
#                                                                 opt.iloc[i]['Strike Price'],
#                                                                 0,
#                                                                 opt.iloc[i]['time_to_expiry']],
#                                                                callPrice=opt.iloc[i]['Close']
#                                                                ).impliedVolatility
#         else:
#             opt.iloc[i, opt.columns.get_loc('IV')] = mibian.BS([opt.iloc[i]['futures_price'],
#                                                                 opt.iloc[i]['Strike Price'],
#                                                                 0,
#                                                                 opt.iloc[i]['time_to_expiry']],
#                                                                putPrice=opt.iloc[i]['Close']
#                                                                ).impliedVolatility
#     return opt


# '''
# Since this process is computationally heavy, we will calculate the implied volatility for first 3 rows and 
# import the remaining implied volatility data from the BankNifty_Preprocessed_Options_Data csv file.
# In BankNifty_Preprocessed_Options_Data csv file, we have already computed and stored the implied volatility data in it.
# '''
# BankNifty_Opt.iloc[:3] = implied_volatility_options(BankNifty_Opt)
opt_IV = pd.read_csv(
    'BankNifty_Preprocessed_Options_Data.csv', index_col=0)
BankNifty_Opt = opt_IV[(opt_IV.Symbol == BankNifty_Opt.Symbol.iloc[0])]
BankNifty_Opt.tail()

,Symbol,Date,Expiry,Option Type,Strike Price,Close,futures_price,time_to_expiry,next_day_close,daily_straddle_pnl,IV,delta
14,BANKNIFTY,2017-12-12,2017-12-28,CE,25200.0,331.60,25178.20,16.0,296.4,-35.2,16.273499,0.496663
16,BANKNIFTY,2017-12-13,2017-12-28,CE,25000.0,393.60,25038.20,15.0,513.2,119.6,18.493652,0.523712
17,BANKNIFTY,2017-12-13,2017-12-28,PE,25000.0,368.40,25038.20,15.0,312.9,-55.5,19.138336,-0.476575
19,BANKNIFTY,2017-12-14,2017-12-28,PE,25200.0,384.55,25213.05,14.0,NaN,NaN,19.856930,-0.486934
18,BANKNIFTY,2017-12-14,2017-12-28,CE,25200.0,391.35,25213.05,14.0,NaN,NaN,19.539356,0.513028


In [71]:
def delta_options(opt):
    opt['delta'] = np.nan
    opt = opt.iloc[:3]
    for i in range(0, len(opt)):
        if opt.iloc[i]['Option Type'] == 'CE':
            opt.iloc[i, opt.columns.get_loc('delta')] = mibian.BS([opt.iloc[i]['futures_price'],
                                                                   opt.iloc[i]['Strike Price'],
                                                                   0,
                                                                   opt.iloc[i]['time_to_expiry']],
                                                                  volatility=opt.iloc[i]['IV']
                                                                  ).callDelta
        else:
            opt.iloc[i, opt.columns.get_loc('delta')] = mibian.BS([opt.iloc[i]['futures_price'],
                                                                   opt.iloc[i]['Strike Price'],
                                                                   0,
                                                                   opt.iloc[i]['time_to_expiry']],
                                                                  volatility=opt.iloc[i]['IV']
                                                                  ).putDelta
    return opt


'''
Since this process is computationally heavy, we will calculate the Delta for first 3 rows and 
the remaining Delta data is already imported from the BankNifty_Preprocessed_Options_Data csv file in the previous step.
'''
BankNifty_Opt.iloc[:3] = delta_options(BankNifty_Opt)
BankNifty_Opt.tail(6)

NameError: name 'norm' is not defined